## Libraries

In [3]:
# processing 
import re
import os
from dotenv import load_dotenv

# embeddings + text
from langchain_core import documents
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import GigaChatEmbeddings

# neo4j
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph

#### .env

In [7]:
load_dotenv()

True

In [9]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
LLM_SCOPE = os.getenv('SCOPE')
LLM_AUTH = os.getenv('AUTH_DATA')

#### .txt

In [ ]:
path = "./закупка.txt"

loader = TextLoader(path, encoding = 'UTF-8')
text_documents = loader.load()

## Text Splitter

In [17]:
class CustomTextSplitter:
    def __init__(self, target_chunk_size=1000):
        
        self.target_chunk_size = target_chunk_size
        self.source = ''
        self.chunks = []
        self.current_chunk = ''
        self.dcts = []
        self.bullet_name = ''
        self.prev_bullet_name = ''
        self.mode_list = ['bullet', 'line', 'space']
        self.sep_dict = {'line': '\n', 'space': ' '}
        self.link_dict = {'bullet': '\n', 'line': '\n', 'space': ' '}
        
    def is_bullet(self, line):
        # Проверка, начинается ли строка с буллита
        return re.match(r'(\d+(\.\d+)*\.\d+)\.? ', line.strip())
    
    def make_chunks(self, input_text, mode):
        link = self.link_dict[mode] 
        if len(input_text) == 0:
            return
        if len(self.current_chunk) > 0:
            # Если текущий чанк не пуст, проверяем, поместится ли туда текущий фрагмент
            new_chunk = self.current_chunk + link + input_text
            if len(new_chunk) <= self.target_chunk_size and mode != 'bullet':
                    self.current_chunk = new_chunk
            else:
                # Сначала сохраняем текущий чанк, потом 
                # в текущий чанк записываем новый фрагмент
                self.chunks.append(self.current_chunk)
                dct = documents.base.Document(self.current_chunk)
                dct.metadata = {'source': self.source}
                # Записываем имя буллита чанка в метаданные
                if mode == 'bullet':
                    dct.metadata['bullet'] = self.prev_bullet_name
                else:
                    dct.metadata['bullet'] = self.bullet_name
                self.dcts.append(dct)
                self.current_chunk = ''
                if len(input_text) <= self.target_chunk_size:
                    self.current_chunk = input_text
                else:
                    # Текст не помещается в чанк целиком. Тогда определяем параметры нового 
                    # разбиения и вызываем рекурсивно этот же метод.
                    next_mode, separator = self.prepare_for_new_split(mode)
                    if next_mode is None:
                        return
                    else:
                        for item in input_text.split(separator):
                            self.make_chunks(item, next_mode)
                        # После завершения разбиения сохраняем текущий чанк, если он не пуст.    
                        if self.current_chunk != '':
                            self.chunks.append(self.current_chunk)
                            dct = documents.base.Document(self.current_chunk)
                            # В текущей реализации в метаданные пишется имя последнего буллита чанка
                            dct.metadata = {'source': self.source,'bullet': self.bullet_name}
                            self.dcts.append(dct)
                            self.current_chunk = ''
        elif len(input_text) <= self.target_chunk_size:
                self.current_chunk = input_text
        else:
            # Текст не помещается в чанк целиком. Тогда определяем параметры нового 
            # разбиения и вызываем рекурсивно этот же метод.            
            next_mode, separator = self.prepare_for_new_split(mode)
            if next_mode is None:
                return
            else:
                for item in input_text.split(separator):
                    self.make_chunks(item, next_mode)
                # После завершения разбиения сохраняем текущий чанк, если он не пуст. 
                if self.current_chunk != '':
                    self.chunks.append(self.current_chunk)
                    dct = documents.base.Document(self.current_chunk)
                    # Записываем имя буллита чанка в метаданные
                    dct.metadata = {'source': self.source,'bullet': self.bullet_name}
                    self.dcts.append(dct)
                    self.current_chunk = ''
    
    def prepare_for_new_split(self, current_mode):
        # Определяем параметры разбиения
            next_mode_index = self.mode_list.index(current_mode) + 1
            if next_mode_index < len(self.mode_list):
                next_mode = self.mode_list[next_mode_index]
                next_separator = self.sep_dict[next_mode]
                return next_mode, next_separator
            else:
                return None, None
            
    def split_text(self, text, source):
        bullet_chunks = []
        current_bullet_chunk = ''
        self.source = source
        # Разбиваем исходный текст на куски, каждый из которых (кроме первого) 
        # начинается с буллита
        lines = text.split("\n")
        for line in lines:
            if len(line) > 0:
                if self.is_bullet(line):
                    if len(current_bullet_chunk) > 0:
                        bullet_chunks.append(current_bullet_chunk)
                    current_bullet_chunk = line      
                elif len(current_bullet_chunk) > 0:
                    current_bullet_chunk = current_bullet_chunk + '\n' + line
                else:
                    current_bullet_chunk = line     
        bullet_chunks.append(current_bullet_chunk)
        # Для каждого буллита сохраняем его имя (=номер) и вызываем метод разбивки на чанки
        for bc in bullet_chunks:
            if len(bc) > 0:
                self.prev_bullet_name = self.bullet_name
                if self.is_bullet(bc):
                    self.bullet_name = self.is_bullet(bc).group()
                else:
                    self.bullet_name = bc.split()[0]
                self.make_chunks(bc, 'bullet')
                
        return self.dcts

In [18]:
# Разбиение текста на чанки. Лимит чанка по умолчанию =1000. Если нужен другой, 
# задаём его с помощью target_chunk_size=... при создании splitter

splitter = CustomTextSplitter()
test_chunks = splitter.split_text(text_documents[0].page_content, text_documents[0].metadata['source'])
test_chunks[:5]

[Document(page_content='Стандарт на процесс Проведение закупки у единственног о поставщика (подрядчика, исполнителя) Содержание 1 Область применения 3 2 Нормативные ссылки 4 3 Термины и сокращения 4 4 Основания для проведения закупки у единственного поставщика (подрядчика, исполнителя) 7 5 Назначение и структура процесса 10 6 Порядок выполнения подпроцесса 03.03.02.01.01 «Закупка у ЕдП (НМЦ менее 10 млн руб. без НДС)» 14 7 Порядок выполнения подпроцесса 03.03.02.01.02 «Закупка у ЕдП (НМЦ 10 млн руб. без НДС и более)» 21 Приложение 1 Описание комплектов документов 31 Приложение 2 Особенности организации закупки у единственного поставщика (подрядчика, исполнителя) по отдельным основаниям Положения о закупках ПАО Компания 1» (обязательное) 32 Библиография 35 История изменений документа 36 Шаблоны, введенные в действие настоящим документом (обеспечивающие выполнение документа) 1 Область применения', metadata={'source': './закупка.txt', 'bullet': 'Стандарт'}),
 Document(page_content='1.1 На

## Neo4j 

### Initialize Nodes with Embeddings

In [10]:
embeddings = GigaChatEmbeddings(credentials=LLM_AUTH, verify_ssl_certs=False)

#### Regular

In [ ]:
# neo4j_vector = Neo4jVector.from_documents(
#     test_chunks, # documents
#     embeddings,
#     url = NEO4J_URI,
#     username = NEO4J_USERNAME,
#     password = NEO4J_PASSWORD
#     # index_name = "vector",  # 'vector' by default
#     # node_label = "Chunk",  # 'Chunk' by default
#     # text_node_property = "content",  # 'text' by default
#     # embedding_node_property = "vector",  # 'embedding' by default
#     # create_id_index = True,  # 'True' by default
# )

#### Hybrid

In [ ]:
# neo4j_vector = Neo4jVector.from_documents(
#     test_chunks, # documents
#     embeddings,
#     url = NEO4J_URI,
#     username = NEO4J_USERNAME,
#     password = NEO4J_PASSWORD, 
#     search_type="hybrid",
#     index_name="Document",  # vector by default
#     node_label="Document"  # Chunk by default
#     # text_node_property="content",  # text by default
#     # embedding_node_property="vector",  # embedding by default
#     # create_id_index=True,  # True by default
# )

###  Add documents

In [ ]:
# Neo4jVector.add_documents()

### Load Existing Nodes / Vectors

#### Regular

In [11]:
neo4j_vector = Neo4jVector.from_existing_index(
    embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="vector", 
    node_label = "Chunk"
)

In [12]:
print(neo4j_vector.index_name)
print(neo4j_vector.node_label)
print(neo4j_vector.embedding_node_property)

vector
Chunk
embedding


#### Hybrid

In [13]:
neo4j_vector_hybrid = Neo4jVector.from_existing_index(
    embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    search_type="hybrid",
    keyword_index_name = "keyword",
    index_name="Document",  # vector by default
    node_label="Document"  # Chunk by default
)

In [15]:
print(neo4j_vector_hybrid.index_name)
print(neo4j_vector_hybrid.keyword_index_name)
print(neo4j_vector_hybrid.node_label)
print(neo4j_vector_hybrid.embedding_node_property)

Document
keyword
Document
embedding


## Quiery

In [23]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [24]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [25]:
cypher = """
  SHOW VECTOR INDEXES
  """
graph.query(cypher)

[{'id': 8,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 17, 21, 4, 49, 896000000, tzinfo=<UTC>),
  'readCount': 8}]

In [26]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """
graph.query(cypher)

[{'count(n)': 1157}]

In [31]:
cypher = """
    MATCH (n:Chunk {bullet: "1.2"})
    RETURN n.text AS text
    """
graph.query(cypher)

[{'text': '1.2. При распространении НМД через механизм тиражирование:\n1.2.1.Положения настоящего стандарта вступают в силу с момента его утверждения и действуют до момента утверждения актуализированной версии стандарта.'},
 {'text': '1.2. Термины и определения'},
 {'text': '1.2. При распространении НМД через механизм тиражирование:'}]